In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/5章'
# os.chdir(path)
# ! pip install japanize-matplotlib

# ５章 分析システムを構築する１０本ノック



### ノック４１：基本的なフォルダを生成しよう

In [4]:
import os 
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "10_output")
master_dir = os.path.join(data_dir, "99_master")
print(input_dir)

data/0_input


In [5]:
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

##### ＊＊必ずデータの配置をおこなってください。＊＊

### ノック４２：入力データのチェック機構を作ろう

In [6]:
import pandas as pd
m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [7]:
tg_ym = "202007"
target_file = "tbl_order_" + tg_ym + ".csv"
target_data = pd.read_csv(os.path.join(input_dir, target_file))

In [8]:
import datetime
max_date = pd.to_datetime(target_data["order_accept_date"]).max()
min_date = pd.to_datetime(target_data["order_accept_date"]).min()
max_str_date = max_date.strftime("%Y%m")
min_str_date = min_date.strftime("%Y%m")
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


In [9]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

def init_tran_df(trg_df):
    trg_df = trg_df.loc[trg_df["store_id"]!=999]
    trg_df = pd.merge(trg_df, m_store, on="store_id", how="left")
    trg_df = pd.merge(trg_df, m_area, on="area_cd", how="left")
    trg_df.loc[trg_df["takeout_flag"]==0, "takeout_name"] = "デリバリー"
    trg_df.loc[trg_df["takeout_flag"]==1, "takeout_name"] = "お持ち帰り"
    trg_df.loc[trg_df["status"]==0, "status_name"] = "受付"
    trg_df.loc[trg_df["status"]==1, "status_name"] = "お支払済み"
    trg_df.loc[trg_df["status"]==2, "status_name"] = "お渡し済"
    trg_df.loc[trg_df["status"]==9, "status_name"] = "キャンセル"
    trg_df.loc[:,"order_date"] = pd.to_datetime(trg_df["order_accept_date"]).dt.date
    trg_df.loc[:,"order_accept_datetime"] = pd.to_datetime(trg_df["order_accept_date"])
    trg_df.loc[:,"delivered_datetime"] = pd.to_datetime(trg_df["delivered_date"])
    trg_df.loc[:,"delta"] = trg_df[["order_accept_datetime","delivered_datetime"]].apply(calc_delta, axis=1)
    return trg_df

target_data = init_tran_df(target_data)


### ノック４３：レポーティング（本部向け）を関数化してみよう

In [11]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Font

def get_rank_df(target_data):
    tmp = target_data.loc[target_data["status"].isin([1,2])]
    rank = tmp.groupby(["store_id"])["total_amount"].sum().sort_value(ascending=False)
    rank = pd.merge(rank, m_store, on="store_id", how="left")
    return rank

In [12]:
def get_cancel_rank_df(target_data):
    cancel_df = pd.DataFrame()
    cancel_cnt = target_data.loc[target_data["status"]==9].groupby(["store_id"])["store_id"].count()
    order_cnt = target_data.loc[target_data["status"].isin([1,2,9])].groupby(["store_id"])["store_id"].count()
    cancel_rate = (cancel_cnt/order_cnt)*100
    cancel_df["cancel_rate"] = cancel_rate
    cance_df = pd.merge(cancel_df, m_store, on="store_id", how="left")
    cance_df = cance_df.sort_values("cancel_rate", ascending=True)
    return cancel_df

In [13]:
def data_export(df, ws, row_start, col_start):
    side = Side(style="thin", color="008080")
    border = Border(top=side, bottom=side, left=side, right=side)
    rows = dataframe_to_rows(df, index=False, header=True)
    for row_no, row in enumerate(rows, row_start):
        for col_no, value in enumerate(row, col_start):
            cell = ws.cell(row_no, col_no)
            cell.value = value
            cell.border = border
            if row_no == row_start:
                cell.fill = PatternFill(patternType="solid", fgColor="008080")
                cell.font = Font(bold=True, color="FFFFFF")

In [ ]:
def make_report_hq(target_data, output_folder):
    rank = get_rank_df(target_data)
    cancel_rank = get_cancel_rank_df(target_data)
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "サマリーレポート（本部向け）"
    cell = ws.cell(1,1)
    cell.value = f'本部向け {max_str_date}月度　サマリーレポート'
    cell.font = Font(bold=True, color="008080", size=20)

    cell = ws.cell(3,2)
    cell.value = f'{max_str_date}月度　売上総額'
    cell.font = Font(bold=True, color="008080",size=20)

    cell = ws.cell(3,6)
    cell.value = f"{'{;,}'.format(rank['total_amount'].sum())}"
    cell.font = Font(bold=True, color="008080", size=20)

    cell = ws.cell(5,2)
    cell.value = f'売上ランキング'
    cell.font = Font(bold=True, color="008080", size=16)

    data_export(rank, ws, 6, 2)

    cell = ws.cell(5,8)
    cell.value = f'キャンセル率ランキング'
    cell.font = Font(bold=True, color="008080", size=16)

    data_export(cancel_rank, ws, 6, 8)

    wb.save(os.path.join(output_folder, f'report_hq_{max_str_date}.xlsx'))
    wb.close()

### ノック４４：レポーティング（店舗向け）を関数化してみよう

### ノック４５：関数を実行し動作を確認してみよう 

### ノック４６：更新に対応できる出力フォルダを作成しよう

### ノック４７：都道府県別で出力できるように出力フォルダを拡張してデータを出力しよう

### ノック４８：前月のデータを動的に読み込もう

### ノック４９：実行して過去データとの比較をしてみよう

### ノック５０：画面から実行できるようにしよう